In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from my_functions import integer_perturbation, get_X_pca_totals, contains_at_least_one, add_new_elements_pca

In [2]:
df = pd.read_excel('data/IncidentiModificato.xlsx', index_col='VERBALE')
df = df.drop(columns=["DATA"])
X = df[['SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'BMI', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro']]
y = df['Mezzo']

df.head()

,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,Testa:Cervelletto,...,Scheletro:Rachide-cervicale,Scheletro:Rachide-toracico,Scheletro:Rachide-lombare,Scheletro:Bacino-e-sacro,Scheletro:Complesso-sterno/claveo/costale,Tot Testa,Tot Torace,Tot Addome,Tot Scheletro,Totale
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,0,81,84.0,1.75,27.428571,0,1,0,0,0,...,0,3,0,3,3,2,0,3,9,14
85829,1,69,69.0,1.62,26.291724,1,4,4,4,4,...,0,0,0,0,4,20,7,1,4,32
85977,1,71,67.0,1.55,27.887617,1,2,0,1,1,...,0,0,0,0,4,6,0,0,4,10
86220,1,54,60.0,1.59,23.733238,1,4,0,0,1,...,0,0,0,0,4,5,3,2,4,14
86247,1,78,69.0,1.67,24.740937,1,2,0,0,0,...,0,0,0,0,4,2,0,2,4,8


In [49]:
from sklearn.manifold import TSNE 

index_continui = df.columns[:5]
index_lesioni = df.columns[6:-5]

X_dati_continui_TSNE = TSNE(n_components=3, perplexity=70, method='exact').fit_transform(df[index_continui])
X_dati_lesioni_TSNE = TSNE(n_components=3, perplexity=70, method='exact').fit_transform(df[index_lesioni])

In [50]:
list_TSNE = []
for i in range(len(X_dati_continui_TSNE)):
    list_TSNE.append(np.concatenate((X_dati_continui_TSNE[i], X_dati_lesioni_TSNE[i])))
df_TSNE = pd.DataFrame(np.array(list_TSNE))

In [51]:
df_TSNE.head()

,0,1,2,3,4,5
0,-34.418255,44.225430,169.446213,-74.489067,-23.254539,116.592781
1,-52.020878,8.928648,-44.019032,-13.119912,-140.731552,-143.876297
2,86.884178,9.404860,-72.032516,99.249184,67.545105,-2.958269
3,-48.753490,-31.268835,7.312100,104.515411,13.877701,-44.132206
4,-11.712326,86.604416,40.180283,29.874331,-22.439728,-32.860069


In [52]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

params = {'hidden_layer_sizes': [[2], [4], [6], [10], [20], [4, 4], [10, 10]],
          'learning_rate_init' : [0.01, 0.2, 0.001],
          'activation': ['identity', 'logistic', 'tanh', 'relu']}

mlp = MLPClassifier(max_iter=5000)

clf = GridSearchCV(mlp, params, n_jobs=-1, cv=5)
scores = cross_val_score(clf, df_TSNE, y, cv=3)
print("\nAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores


Accuracy: 0.45 (+/- 0.03)


array([0.43181818, 0.46511628, 0.44186047])

In [167]:
n = 200
stds = list(df_TSNE.std())

fakes = pd.DataFrame()

for i in range(n):
    sample = df_TSNE.sample()
    for i in sample.columns[:-1]:
        sample[int(i)] += np.random.normal(0, stds[i] * 0.05)
    df_TSNE.loc["FAKE_" + str(i)] = sample
        

ValueError: cannot set a row with mismatched columns

In [168]:
len(df_TSNE)

130

In [162]:
pd.DataFrame()

""


In [155]:
samp

,0,1,2,3,4,5,Mezzo
1,123070.979122,8.928648,-44.019032,-13.119912,-140.731552,-143.876297,1


In [156]:
df_TSNE.append(samp)

,0,1,2,3,4,5,Mezzo
0,-34.418255,44.225430,169.446213,-74.489067,-23.254539,116.592781,0
1,-52.020878,8.928648,-44.019032,-13.119912,-140.731552,-143.876297,1
2,86.884178,9.404860,-72.032516,99.249184,67.545105,-2.958269,1
3,-48.753490,-31.268835,7.312100,104.515411,13.877701,-44.132206,1
4,-11.712326,86.604416,40.180283,29.874331,-22.439728,-32.860069,1
5,45.529079,-35.824352,28.768452,129.089935,-115.585464,-126.771584,1
6,8.400043,-68.013153,-109.934364,162.378281,-0.816307,104.196556,1
7,104.827271,144.176651,-75.457268,29.343359,-64.866638,98.541222,1
8,-95.248848,57.485313,125.670403,95.371559,-8.991525,191.964508,1
9,38.185341,-60.817291,98.293747,-163.522461,-47.747398,-9.635037,1


In [159]:
samp.index = 2

TypeError: Index(...) must be called with a collection of some kind, 2 was passed